In [1]:
%load_ext autoreload

In [2]:
import numpy as np
from functools import partial
from iris_plant_visualizer import IrisPlantVisualizer
import ipywidgets as widgets
from IPython.display import display
from scipy.linalg import block_diag

In [3]:
#pydrake imports
from pydrake.common import FindResourceOrThrow
from pydrake.multibody.parsing import Parser
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.systems.framework import DiagramBuilder
from pydrake.geometry import Role, GeometrySet, CollisionFilterDeclaration
from pydrake.all import RigidTransform, RollPitchYaw, RevoluteJoint
from pydrake.all import RotationMatrix, Rgba
import time
import pydrake.multibody.rational as rational_forward_kinematics
from pydrake.all import RationalForwardKinematics
from pydrake.geometry.optimization import IrisOptions, IrisInRationalConfigurationSpace, HPolyhedron, Hyperellipsoid
from pydrake.solvers import MosekSolver, CommonSolverOption, SolverOptions

In [4]:
from pydrake.geometry.optimization_dev import (CspaceFreePolytope, 
                                               SeparatingPlaneOrder)

# Build and set up the visualization the plant and the visualization of the C-space obstacle

We first set up a simple 2-DOF system and visualize both the plant and the configuration constraint.Click on the two links at the bottom to view the plant and the configuration space.

Note that running this cell multiple times will establish multiple meshcat instances which can fill up your memory. It is a good idea to call "pkill -f meshcat" from the command line before re-running this cell


In [5]:
#construct our robot
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
parser = Parser(plant)
oneDOF_iiwa_asset = FindResourceOrThrow("drake/C_Iris_Examples/assets/oneDOF_iiwa7_with_box_collision.sdf")

box_asset = FindResourceOrThrow("drake/C_Iris_Examples/assets/box_small.urdf")

models = []
models.append(parser.AddModelFromFile(box_asset))
models.append(parser.AddModelFromFile(oneDOF_iiwa_asset, 'right_sweeper'))
models.append(parser.AddModelFromFile(oneDOF_iiwa_asset, 'left_sweeper'))
locs = [[0.,0.,0.],
        [0,1,0.85],
        [0,-1,0.55]]
plant.WeldFrames(plant.world_frame(), 
                 plant.GetFrameByName("base", models[0]),
                 RigidTransform(locs[0]))

t1 = RigidTransform(RollPitchYaw([np.pi/2, 0, 0]).ToRotationMatrix(), locs[1])@RigidTransform(RollPitchYaw([0, 0, np.pi/2]), np.zeros(3))
t2 = RigidTransform(RollPitchYaw([-np.pi/2, 0, 0]).ToRotationMatrix(), locs[2])@RigidTransform(RollPitchYaw([0, 0, np.pi/2]), np.zeros(3))
plant.WeldFrames(plant.world_frame(), 
                 plant.GetFrameByName("iiwa_oneDOF_link_0", models[1]), 
                 t1)
plant.WeldFrames(plant.world_frame(), 
                 plant.GetFrameByName("iiwa_oneDOF_link_0", models[2]), 
                 t2)


plant.Finalize()
idx = 0
q0 = [0.0, 0.0]
val = 1.7
q_low  = np.array([-val, -val])
q_high = np.array([val, val])
# set the joint limits of the plant
for model in models:
    for joint_index in plant.GetJointIndices(model):
        joint = plant.get_mutable_joint(joint_index)
        if isinstance(joint, RevoluteJoint):
            joint.set_default_angle(q0[idx])
            joint.set_position_limits(lower_limits= np.array([q_low[idx]]), upper_limits= np.array([q_high[idx]]))
            idx += 1
        
# construct the RationalForwardKinematics of this plant. This object handles the
# computations for the forward kinematics in the tangent-configuration space
Ratfk = RationalForwardKinematics(plant)

# the point about which we will take the stereographic projections
q_star = np.zeros(plant.num_positions())

do_viz = True

# The object we will use to perform our certification.
cspace_free_polytope = CspaceFreePolytope(plant, scene_graph, SeparatingPlaneOrder.kAffine, q_star)

# This line builds the visualization. Change the viz_role to Role.kIllustration if you
# want to see the plant with its illustrated geometry or to Role.kProximity if you want
# to see the plant with the collision geometries.
visualizer = IrisPlantVisualizer(plant, builder, scene_graph, cspace_free_polytope, viz_role=Role.kIllustration)
visualizer.visualize_collision_constraint(factor = 1.2, num_points = 100)
visualizer.meshcat_cspace.Set2dRenderMode(RigidTransform(RotationMatrix.MakeZRotation(0), np.array([0,0,1])))
visualizer.meshcat_task_space.Set2dRenderMode(RigidTransform(RotationMatrix.MakeZRotation(0), np.array([1,0,0])))

INFO:drake:Meshcat listening for connections at http://localhost:7000
INFO:drake:Meshcat listening for connections at http://localhost:7001


## Set up the sliders so we can move the plant around manually

You can use the sliders below to move the two degrees of freedom of the plant around. A green dot will appear in the TC-space visualization describing the current TC-space configuration.

In [6]:
sliders = []
sliders.append(widgets.FloatSlider(min=q_low[0], max=q_high[0], value=0, description='q0'))
sliders.append(widgets.FloatSlider(min=q_low[1], max=q_high[1], value=0, description='q1'))

q = q0.copy()
def handle_slider_change(change, idx):
    q[idx] = change['new']
    visualizer.show_res_q(q)
    
idx = 0
for slider in sliders:
    slider.observe(partial(handle_slider_change, idx = idx), names='value')
    idx+=1

for slider in sliders:
    display(slider)


Widget Javascript not detected.  It may not be installed or enabled properly. Reconnecting the current kernel may help.


Widget Javascript not detected.  It may not be installed or enabled properly. Reconnecting the current kernel may help.


# Generate and Certify Regions

Around some nominal seed postures, we will grow certified regions by seeding our alternation algorithm using a small initial polytope.

In [7]:
# Some seedpoints
seed_points_q = np.array([   [0.0, 0],
                              [0.7, -0.9],
                              [-0.5, -0.5],
                              [0.4,-1.3]
                              ])
seed_points = np.array([Ratfk.ComputeSValue(seed_points_q[idx], np.zeros((2,)))\
                        for idx in range(seed_points_q.shape[0])])
if do_viz:
    visualizer.plot_cspace_points(seed_points, "/iris_seed_points")
    


default_scale = 1e-2
L1_ball = HPolyhedron.MakeL1Ball(2)
Linf_ball = HPolyhedron.MakeBox(-np.ones(2), np.ones(2))

template_C = np.vstack([L1_ball.A(), Linf_ball.A()])
template_d = np.hstack([default_scale*L1_ball.b(), default_scale/np.sqrt(2)*Linf_ball.b()])


def make_default_polytope_at_point(seed_point):
    return HPolyhedron(template_C, template_d + template_C @ seed_point)


# colors to plot the region. Chosen for color-blind compatibility
colors_dict = {
    0: (144,144,144),
    1:(30,136,229), # bluish
    2: (255, 193, 7), # gold
    3: (0, 140, 6), # green    
}

initial_regions = [make_default_polytope_at_point(s) for s in seed_points]
visualizer.add_group_of_regions_to_visualization(initial_regions, "initial_regions",
                            colors = list(colors_dict.values()),
                            wireframe = False,
                           opacity = 0.2)
visualizer.show_res_q(q_star)

## First we set up some options for our different certification modes

In [17]:
# set up the certifier and the options for different search techniques
solver_options = SolverOptions()
# set this to 1 if you would like to see the solver output in terminal.
solver_options.SetOption(CommonSolverOption.kPrintToConsole, 0)

# The options for when we search for new planes and positivity certificates given the polytopes
find_separation_certificate_given_polytope_options = CspaceFreePolytope.FindSeparationCertificateGivenPolytopeOptions()
find_separation_certificate_given_polytope_options.num_threads = -1
find_separation_certificate_given_polytope_options.verbose = False
find_separation_certificate_given_polytope_options.solver_options = solver_options
find_separation_certificate_given_polytope_options.ignore_redundant_C = True

# The options for when we search for a new polytope given positivity certificates.
find_polytope_given_lagrangian_option = CspaceFreePolytope.FindPolytopeGivenLagrangianOptions()
find_polytope_given_lagrangian_option.solver_options = solver_options
find_polytope_given_lagrangian_option.ellipsoid_margin_cost = CspaceFreePolytope.EllipsoidMarginCost.kGeometricMean
find_polytope_given_lagrangian_option.search_s_bounds_lagrangians = True
# find_polytope_given_lagrangian_option.backoff_scale = 1e-4
find_polytope_given_lagrangian_option.ellipsoid_margin_epsilon = 1e-8


bilinear_alternation_options = CspaceFreePolytope.BilinearAlternationOptions()
bilinear_alternation_options.max_iter = 100
bilinear_alternation_options.convergence_tol = 1e-5
bilinear_alternation_options.find_polytope_options = find_polytope_given_lagrangian_option
bilinear_alternation_options.find_lagrangian_options = find_separation_certificate_given_polytope_options

binary_search_options = CspaceFreePolytope.BinarySearchOptions()
binary_search_options.find_lagrangian_options = find_separation_certificate_given_polytope_options
binary_search_options.scale_min = 1
binary_search_options.scale_max = 50
binary_search_options.max_iter = 50

Clim = np.vstack([np.eye(q_star.shape[0]), -np.eye(q_star.shape[0])])
dlim = np.concatenate([Ratfk.ComputeSValue(q_high, q_star), -Ratfk.ComputeSValue(q_low, q_star)])

# Growing regions with bilinear alternations search

### As the initial regions are fairly small, they won't contain collisions and so can be directly fed into the bilinear alternation algorithm.

In [18]:
# We grow certified regions around each seedpoint using bilinear alternation.
bilinear_alternation_results_by_seed_point = dict.fromkeys([tuple(s) for s in seed_points])
for i, (s, initial_region) in enumerate(zip(seed_points, initial_regions)):
    print(f"starting seedpoint {i+1}/{len(initial_regions)}")
    time.sleep(0.2)
    bilinear_alternation_options.find_polytope_options.s_inner_pts = s
    result = cspace_free_polytope.SearchWithBilinearAlternation(set(), initial_region.A(),
                                                                initial_region.b(),
                                                                bilinear_alternation_options)
    bilinear_alternation_results_by_seed_point[tuple(s)] = [(HPolyhedron(np.vstack([cert.C, Clim]),
                                                                         np.concatenate([cert.d, dlim])), cert) for cert in result]
    

# visualize the regions and corresponding certificates
for i, result in enumerate(bilinear_alternation_results_by_seed_point.values()):
    group_name = f"/bil_alt_seed_point_{i}"
    visualizer.add_group_of_regions_and_certs_to_visualization(result, group_name, 
                                                               wireframe = False, opacity = 0.2)

starting seedpoint 1/4


INFO:drake:det(Q) at the beginning is 4.900500382473016e-05
INFO:drake:Iteration 0: det(Q)=0.00014576238925864208
INFO:drake:Iteration 1: det(Q)=0.0002806159113297737
INFO:drake:Iteration 2: det(Q)=0.0005344000732312085
INFO:drake:Iteration 3: det(Q)=0.0008740376091705029
INFO:drake:Iteration 4: det(Q)=0.0013160775201296712
INFO:drake:Iteration 5: det(Q)=0.0018270475049045126
INFO:drake:Iteration 6: det(Q)=0.002469267808866777
INFO:drake:Iteration 7: det(Q)=0.0032095693243418036
INFO:drake:Iteration 8: det(Q)=0.004083899930157401
INFO:drake:Iteration 9: det(Q)=0.005217228509891534
INFO:drake:Iteration 10: det(Q)=0.0064397033376159055
INFO:drake:Iteration 11: det(Q)=0.007728253756353002
INFO:drake:Iteration 12: det(Q)=0.00933857241785992
INFO:drake:Iteration 13: det(Q)=0.010946345214851256
INFO:drake:Iteration 14: det(Q)=0.01250164908824377
INFO:drake:Iteration 15: det(Q)=0.014376165087566494
INFO:drake:Iteration 16: det(Q)=0.01571596427972976
INFO:drake:Iteration 17: det(Q)=0.016698100

starting seedpoint 2/4


INFO:drake:det(Q) at the beginning is 4.9005001094506195e-05
INFO:drake:Iteration 0: det(Q)=0.00014131240847090125
INFO:drake:Iteration 1: det(Q)=0.0003605858193593601
INFO:drake:Iteration 2: det(Q)=0.0008065910247130536
INFO:drake:Iteration 3: det(Q)=0.0016252957600218611
INFO:drake:Iteration 4: det(Q)=0.003251861538794909
INFO:drake:Iteration 5: det(Q)=0.007590559752036183
INFO:drake:Iteration 6: det(Q)=0.011937531954618595
INFO:drake:Iteration 7: det(Q)=0.012780818683823126
INFO:drake:Iteration 8: det(Q)=0.013219692213514283
INFO:drake:Iteration 9: det(Q)=0.013435386356414988
INFO:drake:Iteration 10: det(Q)=0.013613683193898702
INFO:drake:Iteration 11: det(Q)=0.013794848914524642
INFO:drake:Iteration 12: det(Q)=0.013937560990180167
INFO:drake:Iteration 13: det(Q)=0.014057542582848507
INFO:drake:Iteration 14: det(Q)=0.014145085328447307
INFO:drake:Iteration 15: det(Q)=0.014242816579076289
INFO:drake:Iteration 16: det(Q)=0.014351239088473829
INFO:drake:Iteration 17: det(Q)=0.014468901

starting seedpoint 3/4


INFO:drake:det(Q) at the beginning is 4.900500387407825e-05
INFO:drake:Iteration 0: det(Q)=0.0002893803632078695
INFO:drake:Iteration 1: det(Q)=0.000629240698708994
INFO:drake:Iteration 2: det(Q)=0.0010005786406084907
INFO:drake:Iteration 3: det(Q)=0.0015026235141849568
INFO:drake:Iteration 4: det(Q)=0.002181175468059709
INFO:drake:Iteration 5: det(Q)=0.0030781105752166905
INFO:drake:Iteration 6: det(Q)=0.004126844281648953
INFO:drake:Iteration 7: det(Q)=0.005415459622881643
INFO:drake:Iteration 8: det(Q)=0.0072723018378658836
INFO:drake:Iteration 9: det(Q)=0.010067857816518153
INFO:drake:Iteration 10: det(Q)=0.014442694106815628
INFO:drake:Iteration 11: det(Q)=0.019635337439708356
INFO:drake:Iteration 12: det(Q)=0.02716768240314491
INFO:drake:Iteration 13: det(Q)=0.03960485960130935
INFO:drake:Iteration 14: det(Q)=0.05654903596923884
INFO:drake:Iteration 15: det(Q)=0.07645946700011316
INFO:drake:Iteration 16: det(Q)=0.09267143022313312
INFO:drake:Iteration 17: det(Q)=0.105632308229853

starting seedpoint 4/4


INFO:drake:det(Q) at the beginning is 4.900500574033146e-05
INFO:drake:Iteration 0: det(Q)=0.000507799868097352
INFO:drake:Iteration 1: det(Q)=0.0012460655964163859
INFO:drake:Iteration 2: det(Q)=0.0023452873223409796
INFO:drake:Iteration 3: det(Q)=0.0037317488356948293
INFO:drake:Iteration 4: det(Q)=0.005826506861177007
INFO:drake:Iteration 5: det(Q)=0.012880988574832962
INFO:drake:Iteration 6: det(Q)=0.02039181614958077
INFO:drake:Iteration 7: det(Q)=0.026868254672847088
INFO:drake:Iteration 8: det(Q)=0.03666449217864105
INFO:drake:Iteration 9: det(Q)=0.0468299540714332
INFO:drake:Iteration 10: det(Q)=0.05497347165454071
INFO:drake:Iteration 11: det(Q)=0.06315373408136292
INFO:drake:Iteration 12: det(Q)=0.07181726675898764
INFO:drake:Iteration 13: det(Q)=0.08200086623326285
INFO:drake:Iteration 14: det(Q)=0.09418756353297716
INFO:drake:Iteration 15: det(Q)=0.1095465694699965
INFO:drake:Iteration 16: det(Q)=0.12355858400499432
INFO:drake:Iteration 17: det(Q)=0.13059442591603795
INFO:d

In [28]:
group = [item for i, item in enumerate(bilinear_alternation_results_by_seed_point.values()) if i % 4 == 0][0]
visualizer.add_group_of_regions_and_certs_to_visualization(group, "/final_bil_alt", 
                                                               wireframe = False, opacity = 0.2)

In [26]:
visualizer.meshcat_cspace.Delete("/final_bil_alt")

In [27]:
group

[[(<pydrake.geometry.optimization.HPolyhedron at 0x7fbb9474fb70>,
   <pydrake.geometry.optimization_dev.CspaceFreePolytope.SearchResult at 0x7fbb9475bbf0>),
  (<pydrake.geometry.optimization.HPolyhedron at 0x7fbb9474f7f0>,
   <pydrake.geometry.optimization_dev.CspaceFreePolytope.SearchResult at 0x7fbb9476a930>),
  (<pydrake.geometry.optimization.HPolyhedron at 0x7fbc32b84c30>,
   <pydrake.geometry.optimization_dev.CspaceFreePolytope.SearchResult at 0x7fbb94716df0>),
  (<pydrake.geometry.optimization.HPolyhedron at 0x7fbb91265bb0>,
   <pydrake.geometry.optimization_dev.CspaceFreePolytope.SearchResult at 0x7fbb94719f30>),
  (<pydrake.geometry.optimization.HPolyhedron at 0x7fbb94767930>,
   <pydrake.geometry.optimization_dev.CspaceFreePolytope.SearchResult at 0x7fbb947169f0>),
  (<pydrake.geometry.optimization.HPolyhedron at 0x7fbb9126c530>,
   <pydrake.geometry.optimization_dev.CspaceFreePolytope.SearchResult at 0x7fbb9474c9f0>),
  (<pydrake.geometry.optimization.HPolyhedron at 0x7fbb912

# Growing regions with binary search

### While the bilinear alternation scheme has the flexibility to search for fairly flexible polytopes, it can be relatively slow. We can search for larger regions faster by uniformly growing our polytopes using binary search

In [ ]:
# We grow certified regions around each seedpoint using binary search.
binary_search_results_by_seed_point = dict.fromkeys([tuple(s) for s in seed_points])
for i, (s, initial_region) in enumerate(zip(seed_points, initial_regions)):
    print(f"starting seedpoint {i+1}/{len(initial_regions)}")
    time.sleep(0.2)
    cert = cspace_free_polytope.BinarySearch(set(),initial_region.A(),
                                               initial_region.b(),s,binary_search_options)
    binary_search_results_by_seed_point[tuple(s)] = [(HPolyhedron(cert.C, cert.d), cert)]
                                                     


In [ ]:
# visualize the regions and corresponding certificates
for i, result in enumerate(binary_search_results_by_seed_point.values()):
    group_name = f"/bin_search_seed_point_{i}"
    visualizer.add_group_of_regions_and_certs_to_visualization(result, group_name, 
                                                               wireframe = False, opacity = 0.2)

## Combining Binary Search and Bilinear Alternation
### Of course, we can get the best of both worlds by combining the two methods

In [ ]:
# we use this to back off a bit from the binary search results for numerical reasons
def scale_polytope_about_point(hpoly, s, scale):
    if not hpoly.PointInSet(s):
        raise ValueError(f"The point s must be in the HPolyhedron")
    b_scaled = scale * hpoly.b() + (1 - scale) * hpoly.A() @ s
    return HPolyhedron(hpoly.A(), b_scaled)
    

In [ ]:
# Now we grow certified regions around each seedpoint using bilinear alternation 
# starting from the binary certified regions.

# There is currently a bug in Mosek that kills the kernel if you this cell.

binary_and_bilinear_certified_regions = binary_search_results_by_seed_point.copy()
# for i, (s, region_start) in enumerate(binary_and_bilinear_certified_regions.items()):
#     if i > 2:
#         break
#     print(f"starting seedpoint {i+1}/{len(initial_regions)}")
#     time.sleep(0.2)
#     cur_s = np.array(s)
#     bilinear_alternation_options.find_polytope_options.s_inner_pts = cur_s
    
#     # We back off a little bit from the binary search solution so that we don't encounter numerical issues.
#     initial_region = scale_polytope_about_point(region_start[0][0], cur_s, 0.90)
#     certificates = cspace_free_polytope.SearchWithBilinearAlternation(set(),
#                                                                       initial_region.A(),
#                                                                       initial_region.b(), 
#                                                                       bilinear_alternation_options)
#     binary_and_bilinear_certified_regions[s] += [(HPolyhedron(result.C, result.d), result) for result in certificates]

In [ ]:
# visualize the regions and corresponding certificates
for i, result in enumerate(binary_and_bilinear_certified_regions.values()):
    group_name = f"/bin_then_bil_search_seed_point_{i}"
    visualizer.add_group_of_regions_and_certs_to_visualization(result, group_name, 
                                                               wireframe = False, opacity = 0.2)

## Seeding and Certifying with a Stronger Heuristic
We have also implemented another, strong heuristic for proposing good initial regions based on non-linear optimization. See Appendix TODO of our paper TODO for details

In [ ]:
iris_regions = []
iris_ellipses = []

iris_options = IrisOptions()
iris_options.require_sample_point_is_contained = True
iris_options.configuration_space_margin = 1e-3
iris_options.relative_termination_threshold = 0.001

context_for_iris = visualizer.task_space_diagram_context
for i, s in enumerate(seed_points):
    q = Ratfk.ComputeQValue(s, np.zeros((2,)))
    plant.SetPositions(plant.GetMyMutableContextFromRoot(context_for_iris), q)
    r = IrisInRationalConfigurationSpace(plant, 
                                         plant.GetMyContextFromRoot(context_for_iris),
                                         q_star, iris_options)
    iris_regions.append(r)
    iris_ellipses.append(r.MaximumVolumeInscribedEllipsoid())
    



In [ ]:
visualizer.add_group_of_regions_to_visualization(iris_regions, "/uncertified-iris", 
                                                            wireframe = False, opacity = 0.2)

### These regions tend to be very large, but typically are not completely collision free. We can use the binary search method to find a uniform shrinking of these regions to prove their safety and then again improve them with bilinear alternations.

In [ ]:
binary_search_options_for_iris = CspaceFreePolytope.BinarySearchOptions()
binary_search_options_for_iris.scale_max = 1 # it is highly unlikely that the regions can grow, but we can try
binary_search_options_for_iris.find_lagrangian_options = find_separation_certificate_given_polytope_options
binary_search_options_for_iris.max_iter = 50

In [ ]:
binary_search_region_certificates_for_iris = dict.fromkeys([tuple(s) for s in seed_points])
for i, (s, initial_region) in enumerate(zip(seed_points, iris_regions)):
    print(f"starting seedpoint {i+1}/{len(iris_regions)}")
    time.sleep(0.2)    
    cert = cspace_free_polytope.BinarySearch(set(),
                                                    initial_region.A(),
                                                    initial_region.b(), 
                                                    initial_region.MaximumVolumeInscribedEllipsoid().center(), 
                                                    binary_search_options_for_iris)
    binary_search_region_certificates_for_iris[tuple(s)] = [(HPolyhedron(cert.C, cert.d), cert)]

In [ ]:
for i, result in enumerate(binary_search_region_certificates_for_iris.values()):
    group_name = f"/certified-iris-bin_seed_point_{i}"
    visualizer.add_group_of_regions_and_certs_to_visualization(result, group_name, 
                                                            wireframe = False, opacity = 0.2)

In [ ]:
# Finally we grow the regions again using bilinear alternations.

# There is a numerical bug in Mosek that kills the kernel of we try this
bilinear_certified_regions_iris = binary_search_region_certificates_for_iris.copy()

# There is a numerical bug in Mosek that kills the kernel of we try this.
# for i, (s, region_start) in enumerate(bilinear_certified_regions_iris.items()):
#     print(f"starting seedpoint {i+1}/{len(initial_regions)}")
#     time.sleep(0.2)
#     cur_s = np.array(s)
#     bilinear_alternation_options.find_polytope_options.s_inner_pts = cur_s
    
#     # We back off a little bit from the binary search solution so that we don't encounter numerical issues.
#     initial_region = scale_polytope_about_point(region_start[0][0], cur_s, 0.90)
#     certificates = cspace_free_polytope.SearchWithBilinearAlternation(set(),
#                                                                       initial_region.A(),
#                                                                       initial_region.b(), 
#                                                                       bilinear_alternation_options)
#     bilinear_certified_regions_iris[s] += [(HPolyhedron(result.C, result.d), result) for result in certificates]

In [ ]:
for i, result in enumerate(bilinear_certified_regions_iris.values()):
    visualizer.add_group_of_regions_and_certs_to_visualization(result, "/certified-iris-bin", 
                                                            wireframe = False, opacity = 0.2)

# Now it is time to play with the visualization! If you open the task space visualizer, we can highlight specific geometries and their separating planes in different regions.
## Notice that when regions overlap, we get multiple planes certifying non-collision

In [ ]:
# This plane certifies that the two tips of the flippers don't intersect
visualizer.add_plane_indices_of_interest(118)

# re-display the sliders for convenience
for slider in sliders:
    display(slider)

In [ ]:
# Uncomment this cell for an interface for choosing good planes of interest
# plane_indices = np.arange(0, len(cspace_free_polytope.separating_planes()))
# last_plane_index = 0
# cur_plane_index = 0
# plane_indices_to_geometries = {v: k for k,v in cspace_free_polytope.map_geometries_to_separating_planes().items()}

# while True:
#     visualizer.meshcat_task_space.Delete(f"/plane_{last_plane_index}")
#     geom1, geom2 = plane_indices_to_geometries[cur_plane_index]
#     color = Rgba(0,0,1,1)
    
#     prefix = f"/plane_{cur_plane_index}"
#     visualizer.highlight_geometry_id(geom1, color, prefix+f"/{geom1}")
#     visualizer.highlight_geometry_id(geom2, color, prefix+f"/{geom2}")
#     print(f"current plane index = {cur_plane_index}")
    
#     last_plane_index = cur_plane_index
#     valid_key_pressed = False
#     while not valid_key_pressed:
#         key_press = input("Enter d to increase plane_count, a to decrease, and e to exit...")
#         if key_press in ["a", "d", "e"]:
#             valid_key_pressed = True
#     if key_press == "a":
#         cur_plane_index = max(cur_plane_index - 1, min(plane_indices))
#     elif key_press == "d":
#         cur_plane_index = min(cur_plane_index + 1, max(plane_indices))
#     elif key_press == "e":
#         break
#     print()

In [ ]:
# Uncomment this cell for seeing all the planes associated to a particular geometry id
# interesting_geometry = 92
# plane_indices = np.arange(0, len(cspace_free_polytope.separating_planes()))
# plane_indices_to_geometries = {v: k for k,v in cspace_free_polytope.map_geometries_to_separating_planes().items()}

# associated_plane_indices = []
# for plane_index, (geom1, geom2) in plane_indices_to_geometries.items():
#     if interesting_geometry in (geom1.get_value(), geom2. get_value()):
#         associated_plane_indices.append(plane_index)

        
# last_associated_index_index = 0
# cur_associated_index_index = 0

# visualizer.clear_plane_indices_of_interest()
# while True:
#     last_plane_index = associated_plane_indices[last_associated_index_index]
#     cur_plane_index = associated_plane_indices[cur_associated_index_index]
    
#     print(f"current plane index = {cur_plane_index}")
    
#     visualizer.remove_plane_indices_of_interest(last_plane_index)
#     visualizer.add_plane_indices_of_interest(cur_plane_index)
    
#     cur_q = visualizer.plant.GetPositions(visualizer.plant_context)
#     visualizer.show_res_q(cur_q)
    
#     last_associated_index_index = cur_associated_index_index
#     valid_key_pressed = False
#     while not valid_key_pressed:
#         key_press = input("Enter d to increase plane_count, a to decrease, and e to exit...")
#         if key_press in ["a", "d", "e"]:
#             valid_key_pressed = True
#     if key_press == "a":
#         cur_associated_index_index = max(cur_associated_index_index - 1, min(associated_plane_indices))
#     elif key_press == "d":
#         cur_associated_index_index = min(cur_associated_index_index + 1, max(associated_plane_indices))
#     elif key_press == "e":
#         break
#     print()